## Laboratorium 6

imports

In [1]:
import wikipedia
from wikipedia.exceptions import WikipediaException
import pickle
import csv
import numpy as np
import os
import random
import time
import re
from typing import List, Dict
import string
from gensim.parsing.preprocessing import remove_stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from scipy import sparse


[nltk_data] Downloading package stopwords to /home/hiro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/hiro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Tekst artukułu jest wyszukiwany i pobierany za pomocą biblioteki wikipedia. Po otrzymaniu danego artykułu jest on zapisywany do folderu ```articles``` za pomocą blbioteki ```pickle```.

Załadownie n-pierwszych artykułów z enwiki dump. (Jest to potrzebne ponieważ plik, który zainstalowałem ma ponad 300 000 tytułów artykułu)

In [2]:
class ArticleLoader:
    def __init__(self, n=10**4, file='articles/enwiki-pages-articles.txt'):
        self.n = n
        self.file = file
        self.enwiki_name = 'enwiki-pages-articles.txt'
        self.titles = []
        self.dict_of_key_words = dict()
    # zadanie 1 - przygotowanie zbioru dokumentów tekstowych
    def get_names_of_n_first_articles(self):
        with open(self.file, 'r',encoding="utf8") as f:
            csv_reader = csv.reader(f, delimiter=':')
            i = 0
            for row in csv_reader:
                if i >= self.n:
                    break

                title_of_article = row[-1]
            
                if '/' in title_of_article:
                    continue

                self.titles.append(title_of_article)
                i += 1

        random.shuffle(self.titles)
    
    def load_and_save_articles(self):
        not_found = 0
        if not self.titles:
            self.get_names_of_n_first_articles()

        for title in self.titles:
            try:
                wiki_page = wikipedia.page(title)
                url = wiki_page.url
                content = wiki_page.content
                dict_of_title = {'content': content, 'url': url}
                pickle.dump(dict_of_title, open('articles/{}'.format(title), 'wb'))
            except WikipediaException:
                not_found += 1
            except Exception as e:
                print("exception ocured {}".format(str(e)))
        print("Titles not found: {}".format(not_found))

    # zadabie 2 - określenie słownika słów kluczowych -> będzie to słwonik z kluczem jako nazwa artykułu, a wartością jako tekst artykułu
    
    # stworzenie nowego słownika dla słów kluczowych
    def create_dict_of_key_words(self, length = 1000):
        i = 0
        # self.dict_of_key_words["6"] = pickle.load(open("articles/{}".format(6), "rb"))

        for title in self.pickle_yeild():
            if i > length: break
            if title == self.enwiki_name or "key_words" in title: 
                continue
            
            self.dict_of_key_words[title] = pickle.load(open("articles/{}".format(title), "rb"))
            i+=1


        pickle.dump(self.dict_of_key_words, open('articles/dumped_data/dict_of_key_words_{}'.format(len(self.dict_of_key_words)), 'wb'))
    
    def pickle_yeild(self, path = 'articles'):
        for content in os.listdir(path):
            if os.path.isfile(os.path.join(path, content)):
                yield content

    # wczytanie wcześniej stworzonego słownika dla słów kluczowych
    def load_dict_of_key_words(self, dirpath = 'articles/dumped_data', dumped_file = 'dict_of_key_words_1001'):
        path = os.path.join(dirpath, dumped_file)
        self.dict_of_key_words = pickle.load(open(path, "rb"))




In [3]:
article_handler = ArticleLoader(n= 1000)

In [4]:
article_handler.get_names_of_n_first_articles()

In [7]:
article_handler.load_and_save_articles()

/home/hiro/.local/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/hiro/.local/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Titles not found: 125


In [13]:
article_handler.create_dict_of_key_words()

In [5]:
article_handler.load_dict_of_key_words()

In [6]:
print(article_handler.dict_of_key_words['D. J. Enright']['content'])

Dennis Joseph Enright OBE FRSL (11 March 1920 – 31 December 2002) was a British academic, poet, novelist and critic. He authored Academic Year (1955), Memoirs of a Mendicant Professor (1969) and a wide range of essays, reviews, anthologies, children's books and poems.


== Life ==
Enright was born in Royal Leamington Spa, Warwickshire, to Irish postman father George Enright - a former soldier, "obliged in early life to enlist... as the result of the premature death of his father, a Fenian" - and Welsh chapel-goer mother Grace (née Cleaver); he wrote about his "working-class, Black Country upbringing". Enright stated in his poem "Anglo-Irish" that his "father claimed to be descended from a king called Brian Boru, an ancient hero of Ireland..." but his "mother said that all Irishmen claimed descent from kings but the truth was they were Catholics." His early life was characterised by poverty, the loss of his father, and relationship with his "overworked mother". He was educated at Leamin

In [22]:
class ArticleSearchEngine(object):
    def __init__(self, articles_key_words : Dict[str, Dict[str, str]]):
        self.articles = articles_key_words
        self.titles = list(self.articles.keys())
        self.occured_words = defaultdict(lambda: 0)
        self.uniq_occured_words = None
        self.ids_of_unique_occured_word = None
        self.parsed_article = dict()
        self.stop_words = set(stopwords.words('english'))
        self.lemmatisation = WordNetLemmatizer()
        self.term_by_document_matrix = None
        self.matrix_without_noise = None


    def parse_article_text(self, text: str, if_article = True):                     # dzięki if_article możemy sparsować dostosować parsowanie tekstu w zależności od czy jest artykułem czy nie
        parse_text = text.lower()                                                   # nie ma sensu w odróżnianiu małych i dużych liter
        parse_text = re.sub('['+string.punctuation+']', '', parse_text).split()     # parsowanie po tekście w celu usunięcia niepotrzebnych znajów np. #$%...
        parsed_text = [word for word in parse_text if word not in self.stop_words]  # usuwanie stop_wordów np. 'ours', 'i' ...

        words_dictonary = defaultdict(lambda: 0)                                    # dla nie powstałego słownika przypisz 0 jako wartość
        words_counter = 0                                                           # liczenie ilości słów
        for word in parsed_text:
            lemma_word = self.lemmatisation.lemmatize(word, pos='v')              # pos = 'v' - verb lemmatisation np. goes -> go ...
            words_dictonary[lemma_word] += 1                                        # dzięki defaultdict mamy pewność, że jak nie istnieje dany klucz to stworzy się nowy z wartością 0
            words_counter += 1
            if if_article:
                self.occured_words[lemma_word] += 1
        return dict(words_dictonary = words_dictonary, words_counter = words_counter)
            

    def article_parser(self):
        for article_title in self.titles:
            self.parsed_article[article_title] = dict()
            self.parsed_article[article_title]['article_content'] = self.parse_article_text(self.articles[article_title]['content'])

        self.uniq_occured_words = list(self.occured_words.keys())
        self.ids_of_unique_occured_word = {self.uniq_occured_words[i]: i for i in range(len(self.uniq_occured_words))}


    def create_bags_of_words(self):
        for article_title in self.titles:
            self.parsed_article[article_title]['bag_of_words'] = self.fill_bag_of_words(self.parsed_article[article_title]['article_content'])
    

    def fill_bag_of_words(self, article_content):
        article_words = article_content['words_dictonary'].keys()

        # tworzenie macierzy rzadkiej -> wymiary: ilość wszystkich unikalnych słów we wszystkich tekstów / 1 -> jeden, ponieważ działamy na jednym artykule w danym momencie
        # dok_matrix -> Dictionary Of Keys based sparse matrix, efficient structure for constructing sparse matrices incrementally
        vector = sparse.dok_matrix(np.zeros((len(self.ids_of_unique_occured_word), 1))) 
        for word_key in article_words:
            vector[self.ids_of_unique_occured_word[word_key], 0] = article_content['words_dictonary'][word_key]

        vector /= article_content['words_counter'] # normalizacja wektora -> dzielenie przez ilość unikalnych słów w artykule
        return sparse.csr_matrix(vector) # csr_matrix -> Compressed Sparse Row matrix
    
    
    def create_term_by_document_matrix(self):
        n = len(self.titles)
        m = len(self.uniq_occured_words)
        self.term_by_document_matrix = sparse.lil_matrix((m, n)) # m jest liczbą termów w słowniku, n liczbą dokumentów
        for i in range(n):
            self.term_by_document_matrix[:,i] = self.parsed_article[self.titles[i]]['bag_of_words']
        # utworzenie skompresowanej macierzy rzadkiej by operacje arytmetyczny były wmiarę czasowo wykonalane
        self.term_by_document_matrix = sparse.csr_matrix(self.term_by_document_matrix)


    def multiply_by_IDF(self):
        n = len(self.titles)
        for word in self.uniq_occured_words:
            # nw jest liczbą dokumentów, w których występuje słowo w
            nw = sum(1 for article in self.parsed_article.values() if word in article['article_content']['words_dictonary'])
            id_of_word = self.ids_of_unique_occured_word[word]
            idf_word = np.log(n/nw)
            self.term_by_document_matrix[id_of_word] *= idf_word
    

    def normalize_vector(self, vector):
        return np.sqrt((vector.power(2)).sum())


    def find_best_articles(self, input_words, num_of_articles_to_return = 10, with_noise = True):
        matrix = self.term_by_document_matrix if with_noise else self.matrix_without_noise

        parsed_input_words = self.parse_article_text(input_words, if_article = False)
        q = self.fill_bag_of_words(parsed_input_words)
        q_norm = self.normalize_vector(q)
        probs_of_article = [] # prawdopodobieństwo dla wystąpienia artykułu
        for i in range(len(self.titles)):
            d_i = matrix.getcol(i)
            d_i_norm = self.normalize_vector(d_i)
            cos_theta_i = (q.T @ d_i) / (q_norm * d_i_norm)
            probs_of_article.append((cos_theta_i, i)) # prawdopodowbieństwo oraz miejsce wystąpienia artykułu
        # return: search_link, tytuł, wartość prawdopodobieństwa
        probs_of_article.sort(key = lambda x: x[0], reverse = True)
        
        for prob, idx in probs_of_article[:num_of_articles_to_return]:
            found_article_title = self.titles[idx]
            found_article_url = self.articles[found_article_title]['url']
            print("url: {} title: {} probability: {}".format(found_article_url, found_article_title, prob))
    
    def normalize_vectors(self):
        for i in range(len(self.titles)):
            vector = self.term_by_document_matrix.getcol(i)
            vector_norm = self.normalize_vector(vector)
            self.term_by_document_matrix[:, i] /= vector_norm

    def create_search_engine1(self):
        self.article_parser()
    def create_search_engine2(self):
        self.create_bags_of_words()
    def create_search_engine3(self):
        self.create_term_by_document_matrix()
    def create_search_engine4(self):
        self.multiply_by_IDF()
    def create_search_engine5(self):
        self.normalize_vectors()
        
        
        
        


In [14]:
print(article_engine.parsed_article['Alma Aty']['bag_of_words'])

  (1, 0)	0.0005361930294906167
  (3, 0)	0.0018766756032171583
  (4, 0)	0.0005361930294906167
  (6, 0)	0.0008042895442359249
  (7, 0)	0.0008042895442359249
  (12, 0)	0.0010723860589812334
  (13, 0)	0.0016085790884718498
  (15, 0)	0.00026809651474530834
  (16, 0)	0.0021447721179624667
  (17, 0)	0.00026809651474530834
  (19, 0)	0.00026809651474530834
  (21, 0)	0.00026809651474530834
  (23, 0)	0.0016085790884718498
  (24, 0)	0.00026809651474530834
  (25, 0)	0.0005361930294906167
  (27, 0)	0.00026809651474530834
  (28, 0)	0.0005361930294906167
  (38, 0)	0.00026809651474530834
  (42, 0)	0.00026809651474530834
  (44, 0)	0.0010723860589812334
  (45, 0)	0.00026809651474530834
  (46, 0)	0.0008042895442359249
  (48, 0)	0.00026809651474530834
  (51, 0)	0.0010723860589812334
  (52, 0)	0.00026809651474530834
  :	:
  (187677, 0)	0.00026809651474530834
  (187678, 0)	0.00026809651474530834
  (187679, 0)	0.00026809651474530834
  (187680, 0)	0.00026809651474530834
  (187681, 0)	0.0013404825737265416
  (1

: 

In [45]:
article_engine.parsed_article[' A New Spelling of My Name']['article_content']['words_counter']

716

TESTOWANIE

In [23]:
article_engine = ArticleSearchEngine(article_handler.dict_of_key_words)
article_engine.create_search_engine1()

In [24]:
article_engine.create_search_engine2()


In [25]:
article_engine.create_search_engine3()


In [26]:
article_engine.create_search_engine4()


In [79]:
article_engine.term_by_document_matrix.getcol(1)

<93591x1 sparse matrix of type '<class 'numpy.float64'>'
	with 176 stored elements in Compressed Sparse Row format>

<93591x1 sparse matrix of type '<class 'numpy.float64'>'
	with 136 stored elements in Compressed Sparse Row format>

In [27]:
article_engine.create_search_engine5()


In [50]:
article_engine.find_best_articles("frombork", num_of_articles_to_return = 5)

url: https://en.wikipedia.org/wiki/Meningitis_Now title: Meningitis Trust probability:   (0, 0)	0.6827337428128849
url: https://en.wikipedia.org/wiki/Phonological_history_of_English_vowels title: Vane–vain–vein mergers probability:   (0, 0)	0.13077090848181608
url: https://en.wikipedia.org/wiki/Carbon_Trust title: Carbon Trust probability:   (0, 0)	0.09699294832593375
url: https://en.wikipedia.org/wiki/Fight_for_Sight_(UK) title: Iris Fund for Prevention of Blindness probability:   (0, 0)	0.08675845950882263
url: https://en.wikipedia.org/wiki/Municipal_mergers_and_dissolutions_in_Japan title: Municipal mergers and dissolutions in Japan probability:   (0, 0)	0.07834557180797601


In [62]:
content = "Audre Lorde grows up in Harlem in the 1930s and 1940s, a child of Black West Indian parents. Lorde is legally blind from a very young age, isolating her even further from her surroundings and a family from which she does not receive much warmth or affection. Her two older sisters, Phyllis and Helen, are very close, but are rarely mentioned in Zami and Lorde spends little time with them. Her parents and other adults, especially her mother, discipline her harshly for insolence. Lorde"

content = content.lower()
content2 = content.lower()
print(content)
content = re.sub(r'[^\w\s]', '', content)
print(content)
content = re.sub('[0-9]', '', content)
print(content)
content = re.sub(' {2} +',' ', content)
print(content)
content = remove_stopwords(content)
print(content)

# parsowanie stringów
content2 = re.sub('['+string.punctuation+']', '', content2).split()
print(content2)

# usuowanie stopwords
stop_words = set(stopwords.words('english'))
content2re = [word for word in content2 if word not in stop_words]


print(content2re)
print(content.split())
print(len(content.split()))
print(len(content2re))

audre lorde grows up in harlem in the 1930s and 1940s, a child of black west indian parents. lorde is legally blind from a very young age, isolating her even further from her surroundings and a family from which she does not receive much warmth or affection. her two older sisters, phyllis and helen, are very close, but are rarely mentioned in zami and lorde spends little time with them. her parents and other adults, especially her mother, discipline her harshly for insolence. lorde
audre lorde grows up in harlem in the 1930s and 1940s a child of black west indian parents lorde is legally blind from a very young age isolating her even further from her surroundings and a family from which she does not receive much warmth or affection her two older sisters phyllis and helen are very close but are rarely mentioned in zami and lorde spends little time with them her parents and other adults especially her mother discipline her harshly for insolence lorde
audre lorde grows up in harlem in the